In [35]:
from gurobipy import *
import time
start = time.time()

In [36]:
MAX_N = 96
MAX_T = 6
MAX_F = 3
LARGE_NUM = 10000000

In [37]:
import pandas as pd

In [38]:
file = pd.read_excel("客林專題資料.xlsx", sheet_name='商品相關參數')
f2 = pd.read_excel("客林專題資料.xlsx", sheet_name='非商品相關參數')

### Data Cleaning

In [39]:
#Transfer from unit to package
file['存貨成本'] *= file['package']
file['BO缺貨成本'] *= file['package']
file['LS缺貨成本'] *= file['package']
file['快遞費用'] *= file['package']
file['空運費用'] *= file['package']
file['海運費用'] *= file['package']
file['進貨價格(TWD)'] *= file['package']

# demand / package
file['demand_t'] /= file['package']
file['demand_t+1'] /= file['package']
file['demand_t+2'] /= file['package']
file['demand_t+3'] /= file['package']
file['demand_t+4'] /= file['package']
file['demand_t+5'] /= file['package']

# In-transit inventory / ending inventory / package
file['在途存貨_t+1'] /= file['package']
file['在途存貨_t+2'] /= file['package']
file['期末存貨_t-1'] /= file['package']

### 選取所需欄位、改變欄位名稱

In [40]:
cols = ['商品', 'package', 'backorder percentage', '在途存貨_t+1', '在途存貨_t+2','期末存貨_t-1', 'demand_t', 'demand_t+1', 'demand_t+2', 'demand_t+3','demand_t+4', 'demand_t+5', '存貨成本', 'BO缺貨成本', 'LS缺貨成本', '快遞費用','空運費用',  '海運費用', '可快遞', '可空運', '可海運', '進貨價格(TWD)']
file = file[cols]
ren_col = {'商品':'i', 'package':'P', 'backorder percentage':'beta', '在途存貨_t+1':'Q_t+1', '在途存貨_t+2':'Q_t+2','期末存貨_t-1':'I', 'demand_t':'D_t', 'demand_t+1':'D_t+1', 'demand_t+2':"D_t+2", 'demand_t+3':"D_t+3",'demand_t+4':"D_t+4", 'demand_t+5':"D_t+5", '存貨成本':'Co', 'BO缺貨成本':'Cb', 'LS缺貨成本':'Cl', '快遞費用':'G1','空運費用':"G2",  '海運費用':'G3', '可快遞':"R1", '可空運':"R2", '可海運':"R3", '進貨價格(TWD)':"m"}
file = file.rename(columns=ren_col)

### 資料存放到list，以便後面取用

In [41]:
#fixed cost
K1 = f2.iloc[0, 1]
K2 = f2.iloc[0, 3]
K3 = f2.iloc[0, 5]
K = [0, K1, K2, K3]

#lead time
import math
L1 = math.ceil(f2.iloc[0, 0])
L2 = math.ceil(f2.iloc[0, 2])
L3 = math.ceil(f2.iloc[0, 4])
L = [0, L1, L2, L3]

In [42]:
#variable cost of shipping fee
G = []
G.append([])
for i in range(MAX_N):
    G.append([])
    G[i + 1].append(0)
    for f in range(MAX_F):
        G[i + 1].append(file.iloc[i, 15+f])
        
#Demand        
D = []
D.append([])
for t in range(MAX_T):
    D.append([])
    D[t + 1].append(0)
    for i in range(MAX_N):
        D[t + 1].append(file.iloc[i, 6+t])

#In-transit inventory       
Q = []
Q.append([])
for t in range(1, MAX_T + 1):
    Q.append([])
    Q[t].append(0)
    for i in range(MAX_N):
        if t == 2 or t == 3:
            Q[t].append(file.iloc[i, 1+t])
        else:
            Q[t].append(0)

#Purchase price
M = []
M.append(0)
for i in range(MAX_N):
    M.append(file.iloc[i, 21])

#inventory cost
Co = []
Co.append(0)
for i in range(MAX_N):
    Co.append(file.iloc[i, 12])
    
#lost sales cost
Cb = []
Cb.append(0)
for i in range(MAX_N):
    Cb.append(file.iloc[i, 13])
    
#backorder cost
Cl = []
Cl.append(0)
for i in range(MAX_N):
    Cl.append(file.iloc[i, 14])
    
#backorder percentage
beta = []
beta.append(0)
for i in range(MAX_N):
    beta.append(file.iloc[i, 2])
    
#Initial inventory
I0 = []
I0.append(0)
for i in range(MAX_N):
    I0.append(file.iloc[i, 5])
    
#shipping method
R = []
R.append([])
for i in range(MAX_N):
    R.append([])
    R[i + 1].append(0)
    for f in range(MAX_F):
        R[i + 1].append(file.iloc[i, 18+f])

maxX = []
maxX.append(0)
for i in range(1, MAX_N + 1):
    tmp = 0
    for t in range(1, MAX_T + 1):
        tmp += D[t][i]
        tmp -= Q[t][i]
    tmp -= I0[i] + 1
    maxX.append(max(tmp, 0))

## Model

### 變數設定

In [43]:
eg1 = Model("eg1")  # build a new model, name it as "eg1"

#Decision variables
x, y, z, w = [], [], [], []

x.append([])
for t in range(1, MAX_T + 1):
    x.append([])
    x[t].append(0)
    for i in range(1, MAX_N + 1):
        x[t].append([])
        x[t][i].append(0)
        for f in range(1, MAX_F + 1):
            x[t][i].append(eg1.addVar(lb = 0, vtype = GRB.INTEGER, name = 'x'+str(t)+','+str(i)+','+str(f)))

for t in range(MAX_T + 1):
    y.append([])
    y[t].append(0)
    for i in range(1, MAX_N + 1):
        y[t].append(eg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'y'+str(t)+','+str(i)))

for t in range(MAX_T + 1):
    z.append([])
    z[t].append(0)
    for i in range(1, MAX_N + 1):
        z[t].append(eg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'z'+str(t)+','+str(i)))

w.append([])
for t in range(1, MAX_T + 1):
    w.append([])
    w[t].append(0)
    for f in range(1, MAX_F + 1):
        w[t].append(eg1.addVar(lb = 0, vtype = GRB.BINARY, name = 'w'+str(t)+','+str(f)))

### 目標函數設定

In [44]:
#Objective function
eg1.setObjective(
    quicksum(quicksum(quicksum(((G[i][f] + M[i]) * x[t][i][f]) for f in range(1, MAX_F + 1)) for i in range(1, MAX_N + 1)) for t in range(1, MAX_T + 1))
    + quicksum(quicksum(w[t][f] * K[f] for f in range(1, MAX_F + 1)) for t in range(1, MAX_T + 1))
    + quicksum(quicksum((Co[i] * y[t][i] + beta[i] * Cb[i] * z[t][i] + (1 - beta[i]) * Cl[i] * z[t][i] ) for i in range(1, MAX_N + 1)) for t in range(1, MAX_T + 1))
    , GRB.MINIMIZE)

### 限制式設定

In [45]:
#Constraints
for i in range(1, MAX_N + 1):
    eg1.addConstr(y[0][i] == max(I0[i], 0))

for i in range(1, MAX_N + 1):
    eg1.addConstr(z[0][i] == max(-I0[i], 0))

for t in range(1, MAX_T + 1):
    for i in range(1, MAX_N + 1):
        arrival = 0
        for f in range(1, MAX_F + 1):
            if L[f] < t:
                arrival += x[t - L[f]][i][f]
        eg1.addConstr(y[t][i] - z[t][i] == y[t-1][i] - beta[i] * z[t-1][i] + arrival + Q[t][i] - D[t][i])

# for t in range(1, 2):
#     for i in range(1, MAX_N + 1):
#         eg1.addConstr(y[t][i] - z[t][i] == y[t-1][i] - beta[i] * z[t-1][i] + Q[t][i] - D[t][i])

# for t in range(2, 3):
#     for i in range(1, MAX_N + 1):
#         eg1.addConstr(y[t][i] - z[t][i] == y[t-1][i] - beta[i] * z[t-1][i] + x[1][i][1] + Q[t][i] - D[t][i])
    
# for t in range(3, 4):
#     for i in range(1, MAX_N + 1):
#         eg1.addConstr(y[t][i] - z[t][i] == y[t-1][i] - beta[i] * z[t-1][i] + x[1][i][2] + x[2][i][1] + Q[t][i] - D[t][i])
    
# for t in range(4, MAX_T + 1):
#     for i in range(1, MAX_N + 1):
#         eg1.addConstr(y[t][i] - z[t][i] == y[t-1][i] - beta[i] * z[t-1][i] + x[t-1][i][1] + x[t-2][i][2] + x[t-3][i][3] + Q[t][i] - D[t][i])
    

for t in range(1, MAX_T + 1):
    for i in range(1, MAX_N + 1):
         for f in range(1, MAX_F + 1):
                eg1.addConstr(x[t][i][f] <= R[i][f] * w[t][f] * maxX[i])

In [46]:
# 求解
eg1.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2496 rows, 3090 columns and 5670 nonzeros
Model fingerprint: 0xa1c35bd6
Variable types: 1344 continuous, 1746 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e-02, 6e+02]
  Objective range  [6e-03, 6e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 8e+02]
Found heuristic solution: objective 1.491971e+07
Presolve removed 2323 rows and 2752 columns
Presolve time: 0.01s
Presolved: 173 rows, 338 columns, 632 nonzeros
Found heuristic solution: objective 1.477125e+07
Variable types: 135 continuous, 203 integer (17 binary)

Root relaxation: objective 1.396696e+07, 124 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.3967e+07    0   66 1.4771e+07 1.3967e+07  5.44%     -

### 計算objective value，即總成本

In [47]:
print("objective value =", eg1.objVal)
xs = []
values = []
for var in eg1.getVars():
    if var.x > 0:
        xs.append(var.varName)
        values.append(var.x)
    print(var.varName, '=', var.x)

objective value = 13995571.564213574
x1,1,1 = 0.0
x1,1,2 = 0.0
x1,1,3 = 0.0
x1,2,1 = 0.0
x1,2,2 = 0.0
x1,2,3 = 0.0
x1,3,1 = -0.0
x1,3,2 = -0.0
x1,3,3 = 0.0
x1,4,1 = 0.0
x1,4,2 = 0.0
x1,4,3 = 0.0
x1,5,1 = 0.0
x1,5,2 = 0.0
x1,5,3 = 0.0
x1,6,1 = -0.0
x1,6,2 = 0.0
x1,6,3 = 0.0
x1,7,1 = 0.0
x1,7,2 = 0.0
x1,7,3 = 0.0
x1,8,1 = 0.0
x1,8,2 = 0.0
x1,8,3 = 0.0
x1,9,1 = -0.0
x1,9,2 = 0.0
x1,9,3 = 0.0
x1,10,1 = 0.0
x1,10,2 = 0.0
x1,10,3 = 0.0
x1,11,1 = 0.0
x1,11,2 = 0.0
x1,11,3 = 0.0
x1,12,1 = 0.0
x1,12,2 = 0.0
x1,12,3 = 0.0
x1,13,1 = 0.0
x1,13,2 = 0.0
x1,13,3 = 0.0
x1,14,1 = -0.0
x1,14,2 = 0.0
x1,14,3 = 0.0
x1,15,1 = 0.0
x1,15,2 = 0.0
x1,15,3 = 0.0
x1,16,1 = 0.0
x1,16,2 = 0.0
x1,16,3 = 0.0
x1,17,1 = 3.0
x1,17,2 = 5.0
x1,17,3 = 0.0
x1,18,1 = -0.0
x1,18,2 = 0.0
x1,18,3 = 0.0
x1,19,1 = 0.0
x1,19,2 = 0.0
x1,19,3 = 0.0
x1,20,1 = 0.0
x1,20,2 = 0.0
x1,20,3 = 0.0
x1,21,1 = 0.0
x1,21,2 = 0.0
x1,21,3 = 0.0
x1,22,1 = 0.0
x1,22,2 = 0.0
x1,22,3 = 0.0
x1,23,1 = 0.0
x1,23,2 = 0.0
x1,23,3 = 0.0
x1,24,1 = 0.0
x1,2

x3,10,1 = 0.0
x3,10,2 = 0.0
x3,10,3 = 0.0
x3,11,1 = 0.0
x3,11,2 = 0.0
x3,11,3 = 0.0
x3,12,1 = 0.0
x3,12,2 = 0.0
x3,12,3 = 0.0
x3,13,1 = 0.0
x3,13,2 = 0.0
x3,13,3 = 0.0
x3,14,1 = 0.0
x3,14,2 = 7.0
x3,14,3 = 0.0
x3,15,1 = 0.0
x3,15,2 = 0.0
x3,15,3 = 0.0
x3,16,1 = 0.0
x3,16,2 = 0.0
x3,16,3 = 0.0
x3,17,1 = -0.0
x3,17,2 = 5.0
x3,17,3 = 0.0
x3,18,1 = 0.0
x3,18,2 = 7.0
x3,18,3 = 0.0
x3,19,1 = 0.0
x3,19,2 = 0.0
x3,19,3 = 0.0
x3,20,1 = 0.0
x3,20,2 = 0.0
x3,20,3 = 0.0
x3,21,1 = 0.0
x3,21,2 = 0.0
x3,21,3 = 0.0
x3,22,1 = 0.0
x3,22,2 = 0.0
x3,22,3 = 0.0
x3,23,1 = 0.0
x3,23,2 = 0.0
x3,23,3 = 0.0
x3,24,1 = 0.0
x3,24,2 = 0.0
x3,24,3 = 0.0
x3,25,1 = 0.0
x3,25,2 = 0.0
x3,25,3 = 0.0
x3,26,1 = 0.0
x3,26,2 = 0.0
x3,26,3 = 0.0
x3,27,1 = 0.0
x3,27,2 = 7.0
x3,27,3 = 0.0
x3,28,1 = 0.0
x3,28,2 = 0.0
x3,28,3 = 0.0
x3,29,1 = 0.0
x3,29,2 = 0.0
x3,29,3 = 0.0
x3,30,1 = 0.0
x3,30,2 = 0.0
x3,30,3 = 0.0
x3,31,1 = 0.0
x3,31,2 = 0.0
x3,31,3 = 0.0
x3,32,1 = 0.0
x3,32,2 = 0.0
x3,32,3 = 0.0
x3,33,1 = -0.0
x3,33,2 = -0.0
x3,

x5,12,3 = -0.0
x5,13,1 = 0.0
x5,13,2 = -0.0
x5,13,3 = -0.0
x5,14,1 = -0.0
x5,14,2 = -0.0
x5,14,3 = -0.0
x5,15,1 = 0.0
x5,15,2 = -0.0
x5,15,3 = -0.0
x5,16,1 = 0.0
x5,16,2 = -0.0
x5,16,3 = -0.0
x5,17,1 = -0.0
x5,17,2 = -0.0
x5,17,3 = -0.0
x5,18,1 = -0.0
x5,18,2 = -0.0
x5,18,3 = -0.0
x5,19,1 = 0.0
x5,19,2 = -0.0
x5,19,3 = -0.0
x5,20,1 = 0.0
x5,20,2 = -0.0
x5,20,3 = -0.0
x5,21,1 = 0.0
x5,21,2 = -0.0
x5,21,3 = -0.0
x5,22,1 = 0.0
x5,22,2 = -0.0
x5,22,3 = -0.0
x5,23,1 = 0.0
x5,23,2 = -0.0
x5,23,3 = -0.0
x5,24,1 = 0.0
x5,24,2 = -0.0
x5,24,3 = -0.0
x5,25,1 = 0.0
x5,25,2 = -0.0
x5,25,3 = -0.0
x5,26,1 = 0.0
x5,26,2 = -0.0
x5,26,3 = -0.0
x5,27,1 = -0.0
x5,27,2 = -0.0
x5,27,3 = -0.0
x5,28,1 = 0.0
x5,28,2 = -0.0
x5,28,3 = -0.0
x5,29,1 = 0.0
x5,29,2 = -0.0
x5,29,3 = -0.0
x5,30,1 = 0.0
x5,30,2 = -0.0
x5,30,3 = -0.0
x5,31,1 = 0.0
x5,31,2 = -0.0
x5,31,3 = -0.0
x5,32,1 = 0.0
x5,32,2 = -0.0
x5,32,3 = -0.0
x5,33,1 = -0.0
x5,33,2 = -0.0
x5,33,3 = -0.0
x5,34,1 = 0.0
x5,34,2 = -0.0
x5,34,3 = -0.0
x5,35,1 = 0.

y0,20 = 2.0
y0,21 = 2.0
y0,22 = 3.5
y0,23 = 5.666666666666667
y0,24 = 0.0
y0,25 = 0.0
y0,26 = 0.0
y0,27 = 18.0
y0,28 = 22.0
y0,29 = 0.0
y0,30 = 75.7
y0,31 = 32.85
y0,32 = 99.0
y0,33 = 0.0
y0,34 = 46.95
y0,35 = 12.85
y0,36 = 7.15
y0,37 = 122.15
y0,38 = 41.6
y0,39 = 72.25
y0,40 = 33.833333333333336
y0,41 = 4.0
y0,42 = 102.0
y0,43 = 0.0
y0,44 = 0.0
y0,45 = 5.833333333333333
y0,46 = 0.0
y0,47 = 75.5
y0,48 = 46.0
y0,49 = 27.166666666666668
y0,50 = 9.0
y0,51 = 0.0
y0,52 = 219.16666666666666
y0,53 = 32.0
y0,54 = 756.0
y0,55 = 0.0
y0,56 = 0.0
y0,57 = 0.0
y0,58 = 0.0
y0,59 = 20.0
y0,60 = 20.0
y0,61 = 203.83333333333334
y0,62 = 37.0
y0,63 = 465.0
y0,64 = 0.0
y0,65 = 37.833333333333336
y0,66 = 22.0
y0,67 = 130.0
y0,68 = 0.0
y0,69 = 64.83333333333333
y0,70 = 30.0
y0,71 = 0.0
y0,72 = 342.5
y0,73 = 0.0
y0,74 = 203.83333333333334
y0,75 = 35.0
y0,76 = 112.0
y0,77 = 0.0
y0,78 = 5.0
y0,79 = 1.3333333333333333
y0,80 = 0.0
y0,81 = 0.0
y0,82 = 0.0
y0,83 = 8.0
y0,84 = 0.0
y0,85 = 81.66666666666667
y0,86 = 3

y5,72 = 320.83333333333337
y5,73 = 0.0
y5,74 = 100.83333333333334
y5,75 = 55.0
y5,76 = 0.0
y5,77 = 0.0
y5,78 = 0.1999999999999993
y5,79 = 8.114166666666668
y5,80 = 46.20999999999999
y5,81 = 0.0
y5,82 = 0.4444400800000039
y5,83 = 0.0
y5,84 = 80.0
y5,85 = 13.666666666666668
y5,86 = 18.0
y5,87 = 0.0
y5,88 = 0.0
y5,89 = 0.0
y5,90 = 9.83333333333333
y5,91 = 27.0
y5,92 = 0.0
y5,93 = 40.0
y5,94 = 0.5
y5,95 = 18.999999999999993
y5,96 = 0.0
y6,1 = 0.0
y6,2 = 0.0
y6,3 = 0.0
y6,4 = 9.999999999999998
y6,5 = 1.3877787807814457e-17
y6,6 = 0.0
y6,7 = 0.0
y6,8 = 0.3333333333333317
y6,9 = 0.15000000000000036
y6,10 = 18.0
y6,11 = 154.99999999999997
y6,12 = 0.0
y6,13 = 0.0
y6,14 = 0.0
y6,15 = 28.999999999999993
y6,16 = 0.0
y6,17 = 0.0
y6,18 = 0.125
y6,19 = 33.0
y6,20 = 2.0
y6,21 = 1.9999999999999996
y6,22 = 3.4999999999999996
y6,23 = 5.666666666666668
y6,24 = 0.0
y6,25 = 0.0
y6,26 = 0.0
y6,27 = 0.0
y6,28 = 34.0
y6,29 = 0.0
y6,30 = 11.20000000000001
y6,31 = 14.850000000000001
y6,32 = 98.99999999999999
y6,

z5,27 = 0.0
z5,28 = 0.0
z5,29 = 0.0
z5,30 = 0.0
z5,31 = 0.0
z5,32 = 0.0
z5,33 = 0.01488374999999742
z5,34 = 0.0
z5,35 = 0.0
z5,36 = 0.0
z5,37 = 0.0
z5,38 = 0.0
z5,39 = 0.25
z5,40 = 0.0
z5,41 = 0.0
z5,42 = 0.0
z5,43 = 0.0
z5,44 = 0.0
z5,45 = 0.0
z5,46 = 0.0
z5,47 = 0.0
z5,48 = 0.0
z5,49 = 0.0
z5,50 = 0.0
z5,51 = 0.0
z5,52 = 0.16666666666673713
z5,53 = 0.0
z5,54 = 0.0
z5,55 = 0.0
z5,56 = 0.0
z5,57 = 0.0
z5,58 = 0.0
z5,59 = 0.0
z5,60 = 0.0
z5,61 = 0.0
z5,62 = 0.0
z5,63 = 0.0
z5,64 = 0.0
z5,65 = 0.0
z5,66 = 0.0
z5,67 = 0.0
z5,68 = 0.0
z5,69 = 0.0
z5,70 = 0.0
z5,71 = 0.0
z5,72 = 0.0
z5,73 = 0.0
z5,74 = 0.0
z5,75 = 0.0
z5,76 = 0.0
z5,77 = 0.0
z5,78 = 0.0
z5,79 = 0.0
z5,80 = 0.0
z5,81 = 0.0
z5,82 = 0.0
z5,83 = 0.0
z5,84 = 0.0
z5,85 = 0.0
z5,86 = 0.0
z5,87 = 0.25
z5,88 = 0.0
z5,89 = 0.0
z5,90 = 0.0
z5,91 = 0.0
z5,92 = 0.25000000000000216
z5,93 = 0.0
z5,94 = 0.0
z5,95 = 0.0
z5,96 = 0.0
z6,1 = 0.0
z6,2 = 0.0
z6,3 = 0.0
z6,4 = 0.0
z6,5 = 0.7078666666666666
z6,6 = 0.0
z6,7 = 0.6666666666666666
z6,

### 模型結果result處理，並匯入excel

In [48]:
X = []
V = []
for i in range(len(xs)):
    if xs[i][0] == 'x':
        X.append(xs[i])
        V.append(values[i])

In [49]:
result = pd.DataFrame(columns = ['訂購時間', '訂購商品', '訂購數量', '運送方式'])
for pair in (zip(X, V)):
    decision, value = pair[0], pair[1]
    period, item, shipment = decision.split(',')
    period = period[1:]
    shipTran = {"1":'快遞', "2":'空運', "3":'海運'}
    a = pd.Series([period, item, int(value), shipTran[shipment]], index=['訂購時間', '訂購商品', '訂購數量', '運送方式'])
    result = result.append(a, ignore_index=True)

In [1]:
import os
path = os.getcwd()
result.to_excel(path+"/result.xlsx")

NameError: name 'result' is not defined

In [51]:
end = time.time()
print('建議結果完成，總共耗時', end-start, '秒')

建議結果完成，總共耗時 5.9163618087768555 秒
